In [0]:
%pip install openai

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install azure-search-documents

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
spark.conf.set(
    "fs.azure.account.key.mystore.dfs.core.windows.net",
    "XXXXXX"
)

In [0]:
df = spark.read.text("abfss://rawdata@mystore.dfs.core.windows.net/coverletter.txt")
df.show()

+--------------------+
|               value|
+--------------------+
|Kuldip Mitra     ...|
|                    |
|e-mail: kuldip.mi...|
|LinkedIn:  www.li...|
|Portfolio: https:...|
|Phone: +1(289) 68...|
|Address: Simcoe V...|
|Dear Hiring Manager,|
|I am writing to e...|
|During my 5.5 yea...|
|Additionally, my ...|
|With my unique bl...|
|Thank you for con...|
|        Sincerely,  |
|      Kuldip Mitra  |
+--------------------+



In [0]:
from pyspark.sql.functions import monotonically_increasing_id

docs_df = (
    df
    .withColumnRenamed("value", "document_text")  
    .withColumn("doc_id", monotonically_increasing_id())  
)

docs_df.show(5, truncate=False)


+------------------------------------------------------------------------------------------------------+------+
|document_text                                                                                         |doc_id|
+------------------------------------------------------------------------------------------------------+------+
|Kuldip Mitra                                                                          Date: 27/03/2025|0     |
|                                                                                                      |1     |
|e-mail: kuldip.mitra@ontariotechu.net                                                                 |2     |
|LinkedIn:  www.linkedin.com/in/kuldip-mitra-1a916631                                                  |3     |
|Portfolio: https://kuldipmitra.com/                                                                   |4     |
+------------------------------------------------------------------------------------------------------+

In [0]:
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import ArrayType, StringType

# Function to split text into chunks of chunk_size characters
def chunk_text(text, chunk_size=100):
    if text is None:
        return []
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Register as UDF
chunk_udf = udf(lambda x: chunk_text(x, 100), ArrayType(StringType()))

In [0]:
from pyspark.sql.functions import explode, col

chunked_df = (
    docs_df
    .withColumn("chunks", chunk_udf(col("document_text")))  # apply chunking
    .withColumn("chunk", explode(col("chunks")))  # explode list into separate rows
    .drop("chunks") 
)

chunked_df.show(10, truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+----------------------------------------------------------------------------------------------------+
|document_text                                                                                                                                                                                                                                                                                                                                  

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window = Window.partitionBy("doc_id").orderBy("doc_id")

chunked_df = chunked_df.withColumn("chunk_id", row_number().over(window))
chunked_df.show(10, truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+----------------------------------------------------------------------------------------------------+--------+
|document_text                                                                                                                                                                                                                                                                                                                         

In [0]:
def get_embedding(text):
    if text is None or text.strip() == "":
        return []
    
    response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=text
    )
    return response.data[0].embedding


In [0]:
pdf = chunked_df.select("doc_id", "chunk_id", "chunk").toPandas()
pdf.head(10)


,doc_id,chunk_id,chunk
0,0,1,Kuldip Mitra ...
1,0,2,25
2,2,1,e-mail: kuldip.mitra@ontariotechu.net
3,3,1,LinkedIn: www.linkedin.com/in/kuldip-mitra-1a...
4,4,1,Portfolio: https://kuldipmitra.com/
5,5,1,Phone: +1(289) 688-6062
6,6,1,"Address: Simcoe Village residency, 1910 Simcoe..."
7,7,1,"Dear Hiring Manager,"
8,8,1,I am writing to express my interest in the pos...
9,8,2,oup. With over 17 years of diverse experience ...


In [0]:
OPENAI_ENDPOINT = "https://XXXX.openai.azure.com/"
OPENAI_KEY = "XXXXXXXX"
from openai import AzureOpenAI

EMBEDDING_MODEL = "embed_model"

client = AzureOpenAI(
    api_key=OPENAI_KEY,
    azure_endpoint=OPENAI_ENDPOINT,
    api_version="2024-12-01-preview"
)

In [0]:
import time

embeddings = []

for i, row in pdf.iterrows():
    emb = get_embedding(row["chunk"])
    embeddings.append(emb)
    time.sleep(0.1)  # prevents rate limit

pdf["embedding"] = embeddings


In [0]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

SEARCH_ENDPOINT = "https://XXXXX.search.windows.net"
SEARCH_KEY = "XXXXXXXXX"
INDEX_NAME = "myindexrag"

search_client = SearchClient(
    endpoint=SEARCH_ENDPOINT,
    index_name=INDEX_NAME,
    credential=AzureKeyCredential(SEARCH_KEY)
)



In [0]:
documents = []

for i, row in pdf.iterrows():
    documents.append({
        "id": f"{row['doc_id']}_{row['chunk_id']}",
        "content": row["chunk"],
        "embedding": row["embedding"]
    })


In [0]:
result = search_client.upload_documents(documents)
print("Uploaded:", len(result))


Uploaded: 34


In [0]:
def embed_query(query):
    response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query
    )
    return response.data[0].embedding

query_embedding = embed_query("What are my skills?")


In [0]:
results = search_client.search(
    search_text=None,
    vector_queries=[{
        "kind": "vector",
        "vector": query_embedding,
        "fields": "embedding",
        "k": 5
    }],
    select=["content"]  # only retrievable fields here
)

context = "\n".join([r["content"] for r in results])
print(context)



With my unique blend of IT and engineering roles, coupled with strong leadership and communication s
 welcome the opportunity to discuss further how my skills and experiences align with the needs of yo
ical model development and deployment. I have become skilled in various analytics tools such as Pyth
kills, I am well-prepared to contribute to Steel River Group as Business Analyst Consultant. I would
e developed proficiency in agile software development, data analysis, data visualization, and analyt


In [0]:
OPENAI_ENDPOINT = "https://XXXXXXXX.cognitiveservices.azure.com/"
OPENAI_KEY = "XXXXXXXXXXX"
from openai import AzureOpenAI

EMBEDDING_MODEL = "chat_model"

client = AzureOpenAI(
    api_key=OPENAI_KEY,
    azure_endpoint=OPENAI_ENDPOINT,
    api_version="2024-12-01-preview"
)

In [0]:
rag_prompt = f"""
You are an assistant that answers questions strictly using the provided CONTEXT.
Do not use any external knowledge.
If the answer is not in the context, say "Not found in provided documents."

CONTEXT:
{context}

QUESTION:
What are my skills?
"""


In [0]:
response = client.chat.completions.create(
    model="chat_model",
    messages=[
        {"role": "system", "content": "You answer ONLY from the provided context."},
        {"role": "user", "content": rag_prompt}
    ],
    temperature=0.0   # IMPORTANT: reduce creativity
)

print(response.choices[0].message.content)



Your skills include:

- IT and engineering expertise
- Leadership and communication skills
- Analytical model development and deployment
- Proficiency in analytics tools such as Python
- Agile software development
- Data analysis
- Data visualization
- Analytical skills


